In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [8]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
import numpy as np
import os

In [9]:
#lendo o database
dados =pd.read_csv('databasePJF.csv', header=0, sep=',', quotechar='"')


In [10]:
dados.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [11]:
#criando um dataframe com menos não fraudes para tornar maior a proporção de fraudes para não fraudes.
dadosfraud = dados[(dados.isFraud == 1)]
dadosnfraud = dados[(dados.isFraud == 0)]
#embaralhando o dataframe para ter certeza que os dados não sejam de uma fonte única.
dadosnfraud = dadosnfraud.sample(frac = 1)
#escolhendo aleatoriamente 25.000 dados de fraude
dadosnfraud = dadosnfraud.iloc[0:25000]
#juntando os dados não fraude filtrados e os dados fraude
dadosfinais = pd.concat([dadosfraud,dadosnfraud])
dadosfinais.head()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0
251,1,TRANSFER,2806.0,C1420196421,2806.0,0.0,C972765878,0.0,0.0,1,0
252,1,CASH_OUT,2806.0,C2101527076,2806.0,0.0,C1007251739,26202.0,0.0,1,0
680,1,TRANSFER,20128.0,C137533655,20128.0,0.0,C1848415041,0.0,0.0,1,0


In [12]:
#embaralhando os dataframe selecionado
dadosfinais = dadosfinais.sample(frac = 1)
dadosfinais = dadosfinais.reset_index()
#checando a porcentagem de dados não fraude e fraude
dadosfinais.isFraud.value_counts(True)

0    0.752717
1    0.247283
Name: isFraud, dtype: float64

In [13]:
dadosfinais.columns

Index(['index', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
       'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
       'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [14]:
#utilizando o naive bayes retirado do site https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
#utilizando a mesma base de dados utilizada para treinar o naive bayes.
#selecioando 5.000 dados para teste
dadosteste = dadosfinais[:5000]
#selecionando 20.000 dados para treinamento
dadostreinamento = dadosfinais[5000:]
y = dadostreinamento.isFraud
x = dadostreinamento.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]
z = dadosteste.loc[:, [ 'step', 'amount', 'oldbalanceOrg',
       'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']]

#Iniciando a classificação pelo método de Gauss
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)
#help(model.predict)


[0 1 0 ..., 0 0 0]


In [15]:
#comparando o real com a classficação do naive bayes
dadosteste['predicted'] = pd.Series(predicted)
#selecioando duas colunas
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())
dadosteste.head()

0    3752
1    1248
Name: isFraud, dtype: int64
0    4596
1     404
Name: predicted, dtype: int64


C:\Users\rod_c\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,isFraud,predicted
0,0,0
1,1,1
2,0,0
3,1,0
4,0,0


In [16]:
# com este passo verificamos os verdadeiros e falsos positivos 
br = []

for a in range(len(dadosteste)):
    
    if dadosteste.isFraud[a] == 1:
        
        if dadosteste.predicted[a] == 1:
            br.append('positivos verdadeiros')
            
        if dadosteste.predicted[a] == 0:
            br.append('negativos falsos')
            
    elif dadosteste.isFraud[a] == 0:
        
        if dadosteste.predicted[a] == 0:
            br.append('negativos verdadeiros')
            
        if dadosteste.predicted[a] == 1:
            br.append('positivos falsos')


 fazer falsos positivos, verdadeiros positivos... (proj2)

In [18]:

porcentagem = pd.Series(br)
hue= (porcentagem.value_counts(True))*100
hue

negativos verdadeiros    73.94
negativos falsos         17.98
positivos verdadeiros     6.98
positivos falsos          1.10
dtype: float64

In [19]:
#informação adicional
#fazendo um crosstab entre os tipos de fraude com a quantidade de dados fraudulentos
#com este dado podemos concluir que todos os casos de fraude ocorrem em retiradas de dinheiro ou em tranferencias.
CT = pd.crosstab(dadosfinais.type,dadosfinais.isFraud)
CT

isFraud,0,1
type,,
CASH_IN,5507,0
CASH_OUT,8729,4116
DEBIT,151,0
PAYMENT,8513,0
TRANSFER,2100,4097


Em seguida Foi utilizado o método MultiNomial

In [20]:
# Iniciando a mesma classificação, porém utilizando o método multinoomial. Optamos por classificar novamente para tentar obter
# resultados um pouco melhores.

from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x,y)
predicted = model.predict(z)
print(predicted)

[0 1 0 ..., 1 0 0]


In [21]:
# Comparando valores absolutos entre fraudes e não fraudes da coluna do dataframe original e do que o classificador naive bayes retornou após rodar o método multinomial
dadosteste['predicted'] = pd.Series(predicted)
dadosteste = dadosteste.loc[:, ["isFraud", "predicted"]]
print(dadosteste.isFraud.value_counts())

print(dadosteste.predicted.value_counts())
dadosteste.head()

0    3752
1    1248
Name: isFraud, dtype: int64
0    2531
1    2469
Name: predicted, dtype: int64


,isFraud,predicted
0,0,0
1,1,1
2,0,0
3,1,1
4,0,1


In [22]:
#rodando teste para ver os verdadeiros positivos e falsos positivos do teste multinomial
br = []

for a in range(len(dadosteste)):
    
    if dadosteste.isFraud[a] == 1:
        
        if dadosteste.predicted[a] == 1:
            br.append('positivos verdadeiros')
            
        if dadosteste.predicted[a] == 0:
            br.append('negativos falsos')
            
    elif dadosteste.isFraud[a] == 0:
        
        if dadosteste.predicted[a] == 0:
            br.append('negativos verdadeiros')
            
        if dadosteste.predicted[a] == 1:
            br.append('positivos falsos')

In [23]:
porcentagem = pd.Series(br)
hue= (porcentagem.value_counts(True))*100
hue

negativos verdadeiros    47.14
positivos falsos         27.90
positivos verdadeiros    21.48
negativos falsos          3.48
dtype: float64

CONCLUSÃO
A partir dos dados coletados no teste Gaussiano, que é o teste de variáveis quantitativas, foi possível verificar que a quantidade de dados verdadeiros supera os de falsos de tal maneira obtendo dados acima de 81%. Conforme a captação aleatória de dados reduzidos do naive bayes a resposta se manteve igual. O mesmo resultado se manteve verdadeiro rodando o teste multinomial para variáveis qualitativas, onde obtivemos 68% de acerto no classificador, o que é um número bastante aceitável para um modelo de  naive bayes já pré-estabelecido em uma biblioteca já existente. Os números de positivos verdadeiros no modelo gaussiano foram foram baixos, o que é algo bom, pois como estamos falando de fraude de cartão de crédito, um positivos verdadeiro não é uma coisa boa. No teste multinomial, o número de positivos verdadeiros foi mas alto, mas como o número de positivos falsos foi ainda maior, podemos dizer que não é tão alarmante, pois estes falsos positivos significam que não ocorreu a fraude.

